In [50]:
import numpy as np
import pandas as pd
import requests
from tqdm import tqdm
from glob import glob
import ast, json
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')
#specify home folder (all sub-folders, scripts, data in this folder)
data_folder = ''
api = ''
date = ''

In [47]:
def format_code(code):
    #this function takes in the code, and prepend zeros to the left of the string to make it all 8 characters.
    formatted_code = str(code).zfill(8)
    return formatted_code

In [51]:
##read in table for all rx drugs
kegg_df = pd.DataFrame()
for f in glob(data_folder+'data/kegg_rx_drug_data*'):
  kegg_df = pd.concat([kegg_df, pd.read_csv(f)], axis = 0)
  print(f, kegg_df.shape[0])
kegg_df = kegg_df.drop_duplicates()
kegg_df['japic_code'] = kegg_df['kegg_product_id'].apply(format_code)
print(kegg_df.shape[0])
kegg_df.head(1)

/content/drive/MyDrive/jp_drug_label/data/kegg_rx_drug_data.csv 13382
/content/drive/MyDrive/jp_drug_label/data/kegg_rx_drug_data_v0120.csv 26725
15357


,product,ingredient,indication,kegg_drug_id,kegg_product_id,japic_code
0,オファコルカプセル50mg\n \n \n(レクメド),コール酸;Cholic Acid,先天性胆汁酸代謝異常症治療薬,NaN,70830,00070830


In [ ]:
ades_df = pd.read_csv(data_folder+'data/rx_drug_ade_raw_all_v{}.csv'.format(date))
ades_df['japic_code'] = ades_df['japic_code'].apply(format_code)
ades_df = ades_df.drop_duplicates()
print(ades_df.shape, len(ades_df.japic_code.unique().tolist()))
ades_df.head(1)

---

In [ ]:
#extract only the text
text_strings = []
for i, row in tqdm(ades_df.iterrows()):
  string = BeautifulSoup(row['ade'], 'html.parser')
  if len(string.find_all('table')) > 0:
    string.table.decompose()
    string = string.text.strip()
  else:
    string = None
  text_strings.append(string)
ades_df['ade_text'] = text_strings
ades_df.head(1)

In [30]:
#unique strings
ades_strings = ades_df[ades_df.ade_text.notna()][['ade_text']].drop_duplicates()
ades_strings.to_csv(data_folder+'data/ade/rx_drug_ade_text_v{}.csv'.format(date), index = False)

---

In [ ]:
#extract only the table
tables = pd.DataFrame()
table_strings = []
table_strings_failed = []
for i, row in tqdm(ades_df.iterrows()):
  string = BeautifulSoup(row['ade'], 'html.parser')
  if len(string.find_all('table')) > 0:
    try:
      table = string.find_all('table')[0]
      ades = pd.read_html(str(table))[0]
      ades.columns = ades.iloc[0]
      ades = ades.rename(columns={np.nan: 'category'}).drop(ades.index[0]).reset_index()
      ades['japic_code'] = row['japic_code']
      tables = pd.concat([tables, ades], axis = 0)
    except:
      table_strings_failed.append(row['japic_code'], string.find_all('table')[0])
  else:
    table = None

In [43]:
tables.head(1)

,category,0.1％未満
1,過敏症注）,発疹等
2,皮膚,そう痒感、灼熱感、皮膚潰瘍、接触皮膚炎
3,甲状腺,血中甲状腺ホルモン値（T3、T4値等）の上昇あるいは低下などの甲状腺機能異常
